# Polymorphisms

In [1]:
"""setup"""
from whatever import * 
from toolz.curried import *
from typing import Callable
import toolz
import pytest 
def will_raise(chain, types=TypeError):
    with pytest.raises(types) as t:
        chain.value()

## Chain

The base class to compose fucntions.

In [2]:
@curry
def compare(composed, composition, value):
    assert composed.value(value) == composition(value)

In [3]:
"""test_simple"""
f = Chain(10).range.list
compare(f, compose(list, range), 20)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
f = Chain(10).range(5).list
compare(f, compose(list, partial(range, 5)), 20)
f

[5, 6, 7, 8, 9]

In [5]:
f = Chain(10)[range](5)[list]
compare(f, compose(list, partial(range, 5)))
f

[5, 6, 7, 8, 9]

## Shorthand

In [6]:
f.compose.funcs

(list,)

In [7]:
g = lambda x: x**2
f = _X(10).range(4).map(g).list
compare(f, compose(list, map(g), partial(range, 4)))(20)
f

[16, 25, 36, 49, 64, 81]

In [8]:
g = lambda x: x**2
f = _X(10).range(4) * g | list
compare(f, compose(list, map(g), partial(range, 4)))(20)
f

[16, 25, 36, 49, 64, 81]

In [9]:
g = lambda x: x**2
f = _X(10).range(4) * g | list
compare(f, compose(list, map(g), partial(range, 4)))(20)
f

[16, 25, 36, 49, 64, 81]

In [10]:
g = lambda x: x**2
f = _X(10).range(3) * g + (lambda x: x< 20) | list
compare(f, compose(list, filter(lambda x: x< 20), map(g), partial(range, 3)))(20)
f

[9, 16]

## Complex Arguments

# Lists

In [11]:
f = _X(10).range[[sum, list, len]]
compare(f, compose(list, juxt(sum, list, len), range))(20)
f

[45, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 10]

In [12]:
f = _X(10).range[(sum, list, len)]
# compare(f, compose(juxt(sum, list, len), range))(20)
f._tokens

[[range, [], ()], [(<function sum>, list, <function len>), [], ()]]

In [13]:
f = _X(10).range | (sum, list, len)
compare(f, compose(juxt(sum, list, len), range))(20)
f

(45, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 10)

In [14]:
f = _X(10).range | [sum, list, len]
compare(f, compose(list, juxt(sum, list, len), range))(20)
f

[45, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 10]

In [15]:
_X(10).range[[sum, list, len]] * type * do(print) | list

<class 'int'>
<class 'list'>
<class 'int'>


[<class 'int'>, <class 'list'>, <class 'int'>]

# Dict

In [16]:
_X(10).range[{'a': sum, 'b': list, 'c': len}]

{'a': 45, 'b': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'c': 10}

In [17]:
_X(10).range | {'a': sum, 'b': list, 'c': len}

{'a': 45, 'b': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'c': 10}

# sets

In [18]:
_X(10).range[{sum, list, len}]

{<built-in function len>: 10, <built-in function sum>: 45, <class 'list'>: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [19]:
d = _X(10).range.list | {sum, list, len} | this()[sum].compose
d.value()

45

In [20]:
d = (_X(10).range * range + bool )* {sum, len} | list | list 
d

[{<built-in function len>: 1, <built-in function sum>: 0}, {<built-in function len>: 2, <built-in function sum>: 1}, {<built-in function len>: 3, <built-in function sum>: 3}, {<built-in function len>: 4, <built-in function sum>: 6}, {<built-in function len>: 5, <built-in function sum>: 10}, {<built-in function len>: 6, <built-in function sum>: 15}, {<built-in function len>: 7, <built-in function sum>: 21}, {<built-in function len>: 8, <built-in function sum>: 28}, {<built-in function len>: 9, <built-in function sum>: 36}]

In [21]:
# will_raise(Chain(10).range.map(range).filter(bool).map({'s': sum, 'l': len}).list)

In [22]:
d = (_X(10).range * range + bool ) * {'s': sum, 'l': len} | list 
assert d.f() == _X(10).range.map(range).filter(bool).map({'s': sum, 'l': len}).list.value()

In [23]:
d = (_X(10).range * range + bool ) * [sum, len] | list 
assert d.f() == Chain(10).range.map(range).filter(bool).map(juxt(sum, len)).map(list).list.value()
d

[[0, 1], [1, 2], [3, 3], [6, 4], [10, 5], [15, 6], [21, 7], [28, 8], [36, 9]]

## Values

In [24]:
f = Chain(10).range.list.value()
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [25]:
f = _X(10).range.list.f()
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [26]:
f = _X(10).range.list.value()
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [27]:
_X(10).range.list > identity
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [28]:
f = _X(10) | range | list > identity
assert len(f)==10
assert isinstance(f, list)
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Composition

In [29]:
f = Chain(10).range.list.value
assert isinstance(f, Callable)
f(4)

[0, 1, 2, 3]

In [30]:
f = _X(10).range.list.f
assert isinstance(f, Callable)
f(4)

[0, 1, 2, 3]

In [31]:
f = _X(10).range.list.value
assert isinstance(f, Callable)
f(4)

[0, 1, 2, 3]

In [32]:
f = _X(10).range.list > compose
assert isinstance(f, toolz.functoolz.Compose)
f(4)

[0, 1, 2, 3]

In [33]:
f = _X(10) | range | list > compose
assert isinstance(f, toolz.functoolz.Compose)
f(4)

[0, 1, 2, 3]

# This

In [34]:
class Foo:
    bar = 10
    baz = 'jump'
this(Foo).bar.value()

10

In [35]:
this({'a': Foo, 'b': 'car'})['a'].bar.value()

10

In [36]:
this({'a': Foo, 'b': 'car'})['b'].chain | str.upper | interpose('g') | ''.join

'CgAgR'